In [1]:
#导入数据集预处理、特征工程和模型训练所需的库
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.utils import pad_sequences

In [5]:
trainDFraw=pd.read_csv(".\\dataset\\DB\\DBPEDIA_test.csv")
trainDF = pandas.DataFrame()
trainDF["text"]=trainDFraw.text
trainDF['label'] = trainDFraw.l1

In [6]:
#将数据集分为训练集和验证集
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label编码为目标变量
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [9]:
#加载预先训练好的词嵌入向量
embeddings_index = {}
for i, line in enumerate(open('H:/DATA/wiki-news-300d-1M.vec',encoding='utf-8')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [10]:
#创建一个分词器
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

#将文本转换为分词序列，并填充它们保证得到相同长度的向量
train_seq_x = pad_sequences(token.texts_to_sequences(train_x), maxlen=200)
valid_seq_x = pad_sequences(token.texts_to_sequences(valid_x), maxlen=200)

#创建分词嵌入映射
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [18]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, LSTM, Embedding,Conv2D,Bidirectional,MaxPool2D,Reshape
from keras.models import load_model
from keras.preprocessing.text import Tokenizer,tokenizer_from_json
from keras.callbacks import ModelCheckpoint
import json
import os
import datetime as dt
from matplotlib import pyplot as plt

In [28]:
embedding_dim = 300
vocab_size = len(word_index)
max_len = 200
rnn_units = 100



model = Sequential()
model.add(Embedding(vocab_size+1,embedding_dim,input_length=max_len,weights=[embedding_matrix], trainable=False))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(rnn_units,return_sequences=True),merge_mode='sum'))
model.add(Dropout(0.2))
model.add(Reshape((-1,rnn_units,1)))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(9,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adadelta',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 200, 300)          67998900  
                                                                 
 dropout_15 (Dropout)        (None, 200, 300)          0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 200, 100)         320800    
 nal)                                                            
                                                                 
 dropout_16 (Dropout)        (None, 200, 100)          0         
                                                                 
 reshape_5 (Reshape)         (None, 200, 100, 1)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 198, 98, 64)       640       
                                                      

In [ ]:
epochs = 10
save_dir = './saved_model'
save_fname = os.path.join(save_dir, "主题分析.h5")
callbacks = [ModelCheckpoint(filepath=save_fname, monitor='loss', save_best_only=True)]
model.fit(train_seq_x,train_y,epochs=epochs,validation_data=(valid_seq_x,valid_y),callbacks=callbacks)

Epoch 1/10
1425/1425 [==============================] - 736s 514ms/step - loss: 1.6089 - accuracy: 0.5134 - val_loss: 1.4923 - val_accuracy: 0.5132
Epoch 2/10
1425/1425 [==============================] - 616s 432ms/step - loss: 1.4375 - accuracy: 0.5197 - val_loss: 1.4513 - val_accuracy: 0.5132
Epoch 3/10
1425/1425 [==============================] - 507s 356ms/step - loss: 1.4103 - accuracy: 0.5197 - val_loss: 1.4320 - val_accuracy: 0.5132
Epoch 4/10
1425/1425 [==============================] - 510s 358ms/step - loss: 1.2777 - accuracy: 0.5505 - val_loss: 1.2677 - val_accuracy: 0.5745
Epoch 8/10
1425/1425 [==============================] - 527s 370ms/step - loss: 1.2207 - accuracy: 0.5795 - val_loss: 1.2023 - val_accuracy: 0.5931
Epoch 9/10
1425/1425 [==============================] - 520s 365ms/step - loss: 1.1583 - accuracy: 0.6001 - val_loss: 1.1357 - val_accuracy: 0.6110
Epoch 10/10
1425/1425 [==============================] - 686s 481ms/step - loss: 1.0991 - accuracy: 0.6178 - val